# Lab 7

## Implementing a Naive Bayes Classifier

Today we will be implementing a Naive Bayes Classifier from scratch. 

### Step 1: Data Handling

In [1]:
import csv
def loadCsv(filename):
    lines = csv.reader(open(filename, "rt"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

The above function reads the dataset(csv file) and converts the number strings into floats.

In [2]:
filename = 'pimaindiansdiabetes.data.csv'
dataset = loadCsv(filename)
print(('Loaded data file {0} with {1} rows').format(filename, len(dataset)))

Loaded data file pimaindiansdiabetes.data.csv with 768 rows


Q1. Fill out this function which splits the dataset into a training set and testing set, and returns the two.

In [3]:
import numpy as np
#TODO
def splitDataSet(dataset,splitRatio):
    train_size = int(len(dataset)*splitRatio)    #Size of the training set
    training_set = []  
    copy = list(dataset)
    #Extract training set from dataset using copy of the dataset. 
    while len(training_set) < train_size :
        num = np.random.randint(0,len(copy)-1)
        training_set.append(copy.pop(num))   
    return [training_set,copy]

In [4]:
#This cell should run properly
dataset = [[1], [2], [3], [4], [5]]
splitRatio = 0.67
train, test = splitDataSet(dataset, splitRatio)
print(('Split {0} rows into train with {1} and test with {2}').format(len(dataset), train, test))

Split 5 rows into train with [[3], [2], [4]] and test with [[1], [5]]


### Step 2: Summarize Data

Q2. Fill out this function which separates the data by class(labels). This function returns a dictionary which maps the class to the respective instances.

In [5]:
#TODO
def separatebyClass(dataset):
    separated = {}
    #Fill out this function
    for i in range(len(dataset)):
        vector = dataset[i]
        if(vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [6]:
#This cell should run properly
dataset = [[1,20,1], [2,21,0], [3,22,1]]
separated = separatebyClass(dataset)
print(('Separated instances: {0}').format(separated))

Separated instances: {1: [[1, 20, 1], [3, 22, 1]], 0: [[2, 21, 0]]}


Q3. Fill out functions which return the mean and standard deviation of a list of numbers.

In [7]:
import math
#TODO
def mean(numbers):
    #Fill out this function
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    #Fill out this function 
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [8]:
mean([4,1,1,2,3])

2.2

In [9]:
#This cell should run properly
numbers = [1,2,3,4,5]
print(('Summary of {0}: mean={1}, stdev={2}').format(numbers, mean(numbers), stdev(numbers)))

Summary of [1, 2, 3, 4, 5]: mean=3.0, stdev=1.5811388300841898


Q4. Fill out a function which calculates the mean and standard deviation of each attribute in a class. The mean and standard deviation should be a tuple for each attribute.

In [12]:
#TODO
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [13]:
[i for i in zip(*dataset)]

[(1, 2, 3), (20, 21, 22), (1, 0, 1)]

In [14]:
#This cell should run properly
dataset = [[1,20,0], [2,21,1], [3,22,0]]
summary = summarize(dataset)
print (('Attribute summaries: {0}').format(summary))

Attribute summaries: [(2.0, 1.0), (21.0, 1.0)]


Q5. Fill out the function which summarizes each class. This function returns a dictionary which maps the class to the summary of the attributes of each class.

In [15]:
#TODO
def summarizebyClass(dataset):
    separated = separatebyClass(dataset)
    summaries = {}
    #Fill this out
    for label in separated:
        summaries[label] = summarize(separated[label])
    return summaries


In [16]:
#This should work properly
dataset = [[1,20,1], [2,21,0], [3,22,1], [4,22,0]]
summary = summarizebyClass(dataset)
print(('Summary by class value: {0}').format(summary))

Summary by class value: {1: [(2.0, 1.4142135623730951), (21.0, 1.4142135623730951)], 0: [(3.0, 1.4142135623730951), (21.5, 0.7071067811865476)]}


### Step 3: Make predictions

Q6. Fill out the function which calculates the probability of a Gaussian distribution. 

In [17]:
import math
#TODO
def calculateProbability(x,mean,stdev):
    #Fill this out
    exp = -(x - mean)**2/float(2*(stdev**2))
    return np.exp(exp)/float(np.sqrt(2*np.pi)*stdev)

In [18]:
#This cell should run properly
x = 71.5
mean = 73
stdev = 6.2
probability = calculateProbability(x, mean, stdev)
print(('Probability of belonging to this class: {0}').format(probability))

Probability of belonging to this class: 0.06248965759370005


Q7. Fill out the function which calculates the probability that a data point belongs to either class. We can calculate the probabilities of an attribute belonging to a class using the above function, and we can combine the probabilities by multiplying them(Naive). Thus, this function returns a dictionary which shows the probability that the data summary belongs to a particular class.

In [19]:
#TODO
def calculateClassProbabilities(summaries,inputVector):
    probabilities = {}
    for classValue,classSummaries in summaries.items():
        #Fill this out
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean,stdev = classSummaries[i]
            probabilities[classValue] *= calculateProbability(inputVector[i],mean,stdev)
    return probabilities

In [20]:
#This cell should run properly
summaries = {0:[(1, 0.5)], 1:[(20, 5.0)]}
inputVector = [1.1, '?']
probabilities = calculateClassProbabilities(summaries, inputVector)
print(('Probabilities for each class: {0}').format(probabilities))

Probabilities for each class: {0: 0.78208538795091176, 1: 6.2987362581504424e-05}


Q8a. Fill out the function which makes the prediction that a datapoint belongs to a class.

In [21]:
import operator
#TODO
def predict(summaries,inputVector):
    
    probabilities = calculateClassProbabilities(summaries,inputVector)
    
    #Fill this function out
    return max(probabilities.items(), key=operator.itemgetter(1))[0]


In [22]:
#This cell should run properly
summaries = {'A':[(1, 0.5)], 'B':[(20, 5.0)]}
inputVector = [1.1, '?']
result = predict(summaries, inputVector)
print(('Prediction: {0}').format(result))

Prediction: A


Q8b. Fill out this function for generating predictions for a list of test datapoints.

In [23]:
def getPredictions(summaries,testSet):
    predictions = []
    for i in range(len(testSet)):
        predictions.append(predict(summaries,testSet[i]))
    return predictions

In [24]:
#This cell should run properly
summaries = {'A':[(1, 0.5)], 'B':[(20, 5.0)]}
testSet = [[1.1, '?'], [19.1, '?']]
predictions = getPredictions(summaries, testSet)
print(('Predictions: {0}').format(predictions))

Predictions: ['A', 'B']


### Step 4: Get Accuracy

Q9. Fill out this function which returns the accuracy of the predictions generated by the Naive Bayes Classifier.

In [25]:
#TODO
def getAccuracy(testSet,predictions):
    
    return sum([1 for test,y_pred in zip(testSet,predictions) if test[-1]==y_pred])/float(len(testSet))*100

In [26]:
#This cell should run properly
testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(('Accuracy: {0}').format(accuracy))

Accuracy: 66.66666666666666


### Step 5: Combine it all

Q10. Fill out this Naive Bayes function which takes in the filename and prints its accuracy.

In [27]:
#TODO
def NaiveBayesClassifier(filename):
    dataset = loadCsv(filename)
    splitRatio = 0.67
    train,test = splitDataSet(dataset,splitRatio)
    print(('Split {0} rows into train={1} and test={2} rows').format(len(dataset), len(train), len(test)))
    
    summaries = summarizebyClass(train)
    preds = getPredictions(summaries,test)
    
    return getAccuracy(test,preds)

In [29]:
#This cell should run properly
NaiveBayesClassifier('pimaindiansdiabetes.data.csv')

Split 768 rows into train=514 and test=254 rows


TypeError: 'int' object is not callable